# Spatial joins

In the last lecture we saw how we can merge data frames based on a common attribute value. This type of operation is often referred to as a join in both database and desktop GIS terminology and GeoPandas also uses the term join when referring to a spatial join.

We have seen previously that GeoPandas allows us to use a spatial predicate in many places where a logical expression based on attribute values are called for. The same applies to join operation.

Consider the raptor nest table and the county table.  If we had the name of the county in the raptor nest table we could join the county table to the raptor table on the base of the county name attribute.  We do not have the county name in the raptor table however.  What we have is the geometry of both the raptor nest and the county so we can join the two tables on the basis of the spatial relationship rather than an attribute relationship.

In [ ]:
%matplotlib inline
import geopandas as gpd

raptor = gpd.read_file("data/Raptor_Nests.shp")
county = gpd.read_file("data/colorado_counties.shp")

We perform a spatial join using GeoPandas sjoin method.  Like the Pandas merge method it is called on the GeoPandas object itself and takes a left dataframe and a right dataframe as parameters.  We also have to specify a how parameter which can take the values "inner", "left", and "right". There is no "outer" option for the how parameter in the sjoin method.

With the sjoin method the how parameter determines not only the type of join but which GeoDataFrame's geometry will be used in the resulting GeoDataFrame.  With an inner or left join the left dataframe's geometry will be used.  With a right join the right dataframes geometry will ne used.

Finally we specify the spatial predicate in the op parameter.  Currently the only predicates allowed are "intersects", "contains", and "within"

In [ ]:
raptor_cnty = gpd.sjoin(raptor, county, how="left", op="within")
raptor_cnty

Notice now that the raptor GeoDataFrame contains all the atribute data for the county that contains the point.  If we only want a few of the columns from the county GeoDataFrame we can simply subset that GeoDataFrame by column when we pass it to the sjoin method.

In [ ]:
raptor_cnty = gpd.sjoin(raptor, county[['NAMELSAD10', 'geometry']], how="left", op="within")
raptor_cnty

Notice also that the result includes a index_right column that contains the index of the right GeoDataFrame that satisfied the spatial predicate.

Now we can do things like see how many nests are in each county using basic summary methods.

In [ ]:
import pandas as pd

pd.pivot_table(raptor_cnty, index=['NAMELSAD10', 'recentspec'], values='Nest_ID', aggfunc='count')

Notice also that now we DO have a column in the raptor_cnty table with the county name and so we COULD join with the colorado_county GeoDataFrame on the basis of an attribute relationship rather than a spatial relationship which would be more efficient.

In [ ]:
pd.merge(raptor_cnty, county, how="left", on="NAMELSAD10")

If you expect to join these two tables frequently, especially if the tables are large, then you should consider doing the spatial join once and storing the common field permanently and then do an attribute join when needed in the future.